In [9]:
import glob
import re

In [12]:
def clean_html(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [13]:
email_folder = "tr"
emails = glob.glob(email_folder + "/*")

for email in emails[:2]:
    with open(email, "r") as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
        print(email_clean)

One of a kind Money maker! Try it for free!Content-Type: text/html;
	charset="iso-8859-1"
Content-Transfer-Encoding: 7bit






CONSANTLY being
bombarded by so-called “FREE” money-making systems that teases you with limited
information, and when it’s all said and done, blind-sides you by demanding your
money/credit card information upfront in some slick way, after-the-fact!
Yes, I too was as skeptical about such offers and the Internet in general with
all its hype, as you probably are. Fortunate for me, my main business
slowed-down (I have been self-employed all my life), so I looked for
something to fit my lifestyle and some other way to assist me in paying my
bills, without working myself to death or loosing more money; then, this
proposal to try something new without any upfront investment (great! because
I had none) interested me to click on the link provided. And I don’t regret
at all that I did! I am very happy, and happy enough to recommend it to you as
a system that is true to 